In [1]:
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console
from tools import web_search_tool, save_report_to_md

In [3]:
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
)

In [5]:
research_planner = AssistantAgent(
    "research_planner",
    description="복잡한 질문을 연구 하위 작업으로 분류하는 전략적 연구 코디네이터",
    model_client=model_client,
    system_message="""
            당신은 연구 기획 전문가입니다. 당신의 임무는 집중적인 연구 계획을 수립하는 것입니다.

            각 연구 질문에 대해 다음을 포함하는 집중적인 연구 계획을 수립하십시오.

            1. **핵심 주제**: 연구할 주요 분야 2~3개
            2. **검색어**: 다음을 포함하는 구체적인 검색어 3~5개를 작성하십시오.
            - 최신 동향 및 뉴스
            - 주요 통계 또는 데이터
            - 전문가 분석 또는 연구
            - 미래 전망

            계획은 집중적이고 달성 가능하도록 유지하십시오. 양보다 질을 중시하십시오.""",
)

research_agent = AssistantAgent(
    "research_agent",
    description="콘텐츠를 검색하고 추출하는 웹 리서치 전문가",
    tools=[web_search_tool],
    model_client=model_client,
    system_message="""당신은 웹 리서치 전문가입니다. 리서치 계획에 따라 집중적인 검색을 수행하는 것이 당신의 임무입니다.

            리서치 전략:
            1. 리서치 계획에 따라 **3~5건의 검색을 실행합니다.**
            2. 결과에서 **핵심 정보 추출**:
            - 주요 사실 및 통계
            - 최근 동향
            - 전문가 의견
            - 중요한 맥락

            3. **품질 중심**:
            - 권위 있는 출처를 우선시합니다.
            - 최근 정보(2년 이내)를 찾습니다.
            - 다양한 관점을 기록합니다.

            계획에 따라 검색을 완료한 후, 찾은 내용을 요약합니다. 목표는 5~10개의 양질의 출처를 수집하는 것입니다.""",
)

research_analyst = AssistantAgent(
    "research_analyst",
    description="연구 보고서를 작성하는 전문 분석가",
    model_client=model_client,
    system_message="""연구 분석가로서, 수집된 연구를 바탕으로 종합적인 보고서를 작성하세요.

            다음 내용을 포함하여 연구 보고서를 작성하세요.

            ## 요약
            - 주요 결과 및 결론
            - 주요 통찰력

            ## 배경 및 현황
            - 현재 상황
            - 최근 동향
            - 주요 통계 및 데이터

            ## 분석 및 통찰력
            - 주요 동향
            - 다양한 관점
            - 전문가 의견

            ## 미래 전망
            - 새로운 동향
            - 예측
            - 시사점

            ## 출처
            - 사용된 모든 출처 나열

            수집된 연구를 바탕으로 명확하고 체계적인 보고서를 작성하세요. 작성이 완료되면 "REPORT_COMPLETE"로 보고서를 마무리하세요.""",
)

quality_reviewer = AssistantAgent(
    "quality_reviewer",
    description="연구의 완성도와 정확성을 평가하는 품질 보증 전문가",
    tools=[save_report_to_md],
    model_client=model_client,
    system_message="""당신은 훌륭한 리뷰어입니다. 당신의 임무는 연구 분석가가 완전한 연구 보고서를 작성했는지 확인하는 것입니다.

            다음 사항을 확인하세요.
            - 연구 분석가가 작성한 "REPORT_COMPLETE"로 끝나는 포괄적인 연구 보고서
            - 연구 질문에 대한 완전한 답변
            - 출처가 인용되었고 신뢰할 수 있음
            - 보고서에 요약, 주요 정보, 분석 및 출처가 포함되어 있음

            "REPORT_COMPLETE"로 끝나는 완전한 연구 보고서가 있는 경우:
            1. 먼저 save_report_to_md 도구를 사용하여 보고서를 report.md에 저장합니다.
            2. 그런 다음 "연구가 완료되었습니다. 보고서가 report.md에 저장되었습니다. 보고서를 검토하시고 승인 또는 추가 연구가 필요한지 알려주십시오."라고 말합니다.

            연구 분석가가 아직 완전한 보고서를 작성하지 않았다면 지금 작성하도록 안내하세요.""",
)

research_enhancer = AssistantAgent(
    "research_enhancer",
    description="중요한 격차만을 식별하는 전문가",
    model_client=model_client,
    system_message="""연구 개선 전문가로서, 핵심적인 결함만 파악하는 것이 당신의 역할입니다.
            
            연구를 검토하고 다음과 같은 중대한 결함이 있는 경우에만 추가 검색을 제안해 주십시오.
            - 최근 6개월간의 연구 결과가 전혀 누락된 경우
            - 통계나 데이터가 전혀 없는 경우
            - 특별히 요청된 중요한 관점이 누락된 경우
            
            연구가 기본적인 내용을 어느 정도 잘 다루고 있다면, "보고서 작성에 충분한 연구 결과입니다."라고 말하십시오.
            
            절대적으로 필요한 경우에만 1~2건의 추가 검색을 제안해 주십시오. 완벽한 보고서 작성보다는 양질의 보고서를 작성하는 것을 우선시합니다.""",
)

user_proxy = UserProxyAgent(
    "user_proxy",
    description="추가 연구를 요청하거나 최종 결과를 승인할 수 있는 인간 검토자",
    input_func=input,
)

In [6]:
selector_prompt = """
    대화 기록을 기반으로 현재 작업에 가장 적합한 에이전트를 선택하세요.
    
    {roles}
    
    현재 대화:
    {history}
    
    사용 가능한 에이전트:
    - research_planner: 연구 접근 방식 계획(시작 시에만)
    - research_agent: 웹 소스에서 콘텐츠 검색 및 추출(계획 후)
    - research_enhancer: 중요한 차이점만 파악(use sparingly)
    - research_analyst: 최종 연구 보고서 작성
    - quality_reviewer: 완전한 보고서가 있는지 확인
    - user_proxy: 담당자에게 피드백 요청
    
    워크플로:
    1. 아직 계획이 완료되지 않은 경우 → research_planner 선택
    2. 계획은 완료되었지만 연구가 완료되지 않은 경우 → research_agent 선택
    3. research_agent가 초기 검색을 완료한 후 → research_enhancer 한 번 선택
    4. enhancer가 "진행해도 됨"이라고 말하면 → research_analyst 선택
    5. enhancer가 중요한 검색을 제안하면 → research_agent를 research_analyst보다 한 번 더 선택
    6. research_analyst가 "보고서 완료" → quality_reviewer 선택
    7. quality_reviewer가 사용자 피드백을 요청한 경우 → user_proxy 선택
    
    중요: research_agent가 최대 2회 검색한 후에는 research_analyst로 진행하세요.
    
    이 워크플로에 따라 다음에 작업할 에이전트를 선택하세요."""

In [7]:
text_termination = TextMentionTermination("APPROVED")
max_message_termination = MaxMessageTermination(max_messages=50)
termination_condition = text_termination | max_message_termination

team = SelectorGroupChat(
    participants=[
        research_agent,
        research_analyst,
        research_enhancer,
        research_planner,
        quality_reviewer,
        user_proxy,
    ],
    selector_prompt=selector_prompt,
    model_client=model_client,
    # allow_repeated_speaker=True,
    termination_condition=termination_condition,
)

In [8]:
await Console(
    team.run_stream(task="2026년 주식 유망종목"),
)

---------- TextMessage (user) ----------
2026년 주식 유망종목
---------- TextMessage (research_planner) ----------
### 연구 계획: 2026년 주식 유망종목

1. **핵심 주제**
   - **성장 산업**: 친환경 에너지, 인공지능(AI) 및 제약 산업
   - **시장 동향**: 주요 주식 시장 지수 및 경제 지표의 변동
   - **투자 전략**: 가치 투자와 성장 투자 분석

2. **검색어**
   - "2026년 유망 주식 산업별 최신 동향 및 뉴스"
   - "2026년 주식 시장 주요 통계 및 데이터"
   - "전문가 분석 2026년 주식 유망종목 추천"
   - "2026년 주식 시장 미래 전망과 투자 전략"
   - "친환경 에너지 및 AI 주식 데이터 및 예측"

이 연구 계획은 향후 3년 내 주식 시장의 흐름을 이해하고, 투자 결정을 내리는 데 도움을 줄 수 있는 정보를 체계적으로 수집하고 분석하는 것을 목표로 합니다. 각 검색어에 대한 자료를 신중하게 선택하여 유망 종목을 파악하고, 특정 산업의 성장 가능성을 평가하는 데 중점을 두어야 합니다.
---------- ToolCallRequestEvent (research_agent) ----------
[FunctionCall(id='call_MyfrJGLftCe1hyZPz6IWEl8l', arguments='{"query": "2026년 유망 주식 산업별 최신 동향 및 뉴스"}', name='web_search_tool'), FunctionCall(id='call_hwiAqyRRegmrt0BYBXjwDs1W', arguments='{"query": "2026년 주식 시장 주요 통계 및 데이터"}', name='web_search_tool'), FunctionCall(id='call_08wxaP1ImnzfYzlTk5oIQCzx', arguments='{"query": "전문가 분석 2026년 주식 유망종목 

Error processing publish message for SelectorGroupChatManager_58479edc-5cd0-4c31-bfd2-eafb3afc1723/58479edc-5cd0-4c31-bfd2-eafb3afc1723
Traceback (most recent call last):
  File "/Users/junghun/code/AI_Agent/autogen/deep-search-clone/.venv/lib/python3.13/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
    )
    ^
  File "/Users/junghun/code/AI_Agent/autogen/deep-search-clone/.venv/lib/python3.13/site-packages/autogen_core/_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/junghun/code/AI_Agent/autogen/deep-search-clone/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

RuntimeError: BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 128000 tokens. However, your messages resulted in 192025 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Traceback:
Traceback (most recent call last):

  File "/Users/junghun/code/AI_Agent/autogen/deep-search-clone/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_base_group_chat_manager.py", line 164, in handle_agent_response
    await self._transition_to_next_speakers(ctx.cancellation_token)

  File "/Users/junghun/code/AI_Agent/autogen/deep-search-clone/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_base_group_chat_manager.py", line 176, in _transition_to_next_speakers
    speaker_names = await speaker_names_future
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/junghun/code/AI_Agent/autogen/deep-search-clone/.venv/lib/p